In [15]:
import cv2
import numpy as np           
import csv
import pandas as pd

text = str('grapeleaves_2.jpg')
img = cv2.imread(text)
original = img.copy()
neworiginal = img.copy() 
cv2.imshow('original',img)

#Guassian blur
blur1 = cv2.GaussianBlur(img,(3,3),1)
cv2.imshow('gaussian blur',blur1)

#mean-shift algo
newimg = np.zeros((img.shape[0], img.shape[1],3),np.uint8)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER , 10 ,1.0)

img = cv2.pyrMeanShiftFiltering(blur1, 20, 30, newimg, 0, criteria)
cv2.imshow('means shift image',img)

#Guassian blur
blur = cv2.GaussianBlur(img,(11,11),1)

#Canny-edge detection
canny = cv2.Canny(blur, 160, 290)
canny = cv2.cvtColor(canny,cv2.COLOR_GRAY2BGR)

bordered = cv2.cvtColor(canny,cv2.COLOR_BGR2GRAY)
_,contours,hierarchy = cv2.findContours(bordered, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

maxC = 0
for x in range(len(contours)):
    if len(contours[x]) > maxC:
        maxC = len(contours[x])
        maxid = x

perimeter = cv2.arcLength(contours[maxid],True)
#print perimeter
Tarea = cv2.contourArea(contours[maxid])
cv2.drawContours(neworiginal,contours[maxid],-1,(0,0,255))
cv2.imshow('Contour',neworiginal)

height, width, _ = canny.shape
min_x, min_y = width, height
max_x = max_y = 0
frame = canny.copy()

for contour, hier in zip(contours, hierarchy):
    (x,y,w,h) = cv2.boundingRect(contours[maxid])
    min_x, max_x = min(x, min_x), max(x+w, max_x)
    min_y, max_y = min(y, min_y), max(y+h, max_y)
    if w > 80 and h > 80:
        #cv2.rectangle(frame, (x,y), (x+w,y+h), (255, 0, 0), 2)   #we do not draw the rectangle as it interferes with contour later on
        roi = img[y:y+h , x:x+w]
        originalroi = original[y:y+h , x:x+w]

if (max_x - min_x > 0 and max_y - min_y > 0):
    roi = img[min_y:max_y , min_x:max_x]
    originalroi = original[min_y:max_y , min_x:max_x]
    #cv2.rectangle(frame, (min_x, min_y), (max_x, max_y), (255, 0, 0), 2)   #we do not draw the rectangle as it interferes with contour

cv2.imshow('ROI', frame)
cv2.imshow('rectangle ROI', roi)
img = roi

#Changing colour-space
imghls = cv2.cvtColor(roi, cv2.COLOR_BGR2HLS)
cv2.imshow('HSI', imghls)
imghls[np.where((imghls==[30,200,2]).all(axis=2))] = [0,200,0]

huehls = imghls[:,:,0]
cv2.imshow('img_hue hls',huehls)
huehls[np.where(huehls==[0])] = [35]

ret, thresh = cv2.threshold(huehls,28,255,cv2.THRESH_BINARY_INV)
cv2.imshow('thresh', thresh)

mask = cv2.bitwise_and(originalroi,originalroi,mask = thresh)
cv2.imshow('masked out img',mask)

#Finding contours for all infected regions
_,contours,heirarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

Infarea = 0
for x in range(len(contours)):
    cv2.drawContours(originalroi,contours[x],-1,(0,0,255))
    cv2.imshow('Contour masked',originalroi)
    
#Calculating area of infected region
    Infarea += cv2.contourArea(contours[x])

if Infarea > Tarea:
    Tarea = img.shape[0]*img.shape[1]

print ('_________________________________________\n Perimeter: %.2f' %(perimeter) 
       + '\n_________________________________________')
print ('_________________________________________\n Infected area: %.2f' %(Infarea) 
       + '\n_________________________________________')

print ('_________________________________________\n Total area: %.2f' %(Tarea) 
       + '\n_________________________________________')

#Finding the percentage of infection in the leaf

try:
    per = 100 * Infarea/Tarea
except ZeroDivisionError:
    per = 0

print ('_________________________________________\n Percentage of infection region: %.2f' %(per) 
       + '\n_________________________________________')

#import csv file library 
import csv

directory = r'C:/Users/USER/Desktop/skripsi'
filename = directory +'/data_uji.csv' 
imgid = "/".join(text.split('/')[-2:])

fieldnames = ['fold num', 'imgid', 'feature1', 'feature2', 'feature3']
print ('Appending to ' + str(filename)+ '...')
         
try:
        log = pd.read_csv(filename)
        logfn = pd.to_numeric(log.tail(1)['fold num'], errors='coerce')
        foldnum = (logfn+1)%10
        L = [str(foldnum), imgid, str(Tarea), str(Infarea), str(perimeter)]
        my_df = pd.DataFrame([L])
        my_df.to_csv(filename, mode='a', index=False, header=False)
        print ('\nFile ' + str(filename)+ ' updated!' )
                

except IOError:
        if directory not in os.listdir():
            os.system('mkdir ' + directory)

        foldnum = 0
        L = [str(foldnum), imgid, str(Tarea), str(Infarea), str(perimeter)]

        my_df = pd.DataFrame([fieldnames, L])
        my_df.to_csv(filename, index=False, header=False)
        print ('\nFile ' + str(filename)+ ' updated!' )

cv2.waitKey(0)
cv2.destroyAllWindows()

_________________________________________
 Perimeter: 1347.63
_________________________________________
_________________________________________
 Infected area: 1941.50
_________________________________________
_________________________________________
 Total area: 43036.00
_________________________________________
_________________________________________
 Percentage of infection region: 4.51
_________________________________________
Appending to C:/Users/USER/Desktop/skripsi/data_uji.csv...

File C:/Users/USER/Desktop/skripsi/data_uji.csv updated!


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# reading csv file and extracting class column to y.
data = pd.read_csv("dataset_grapeleaves.csv")

# extracting two features
X = data.drop(['imgid','fold num'], axis=1)
y = X['label']
X = X.drop('label', axis=1)
print("\nTraining dataset:-\n")
print(X)

log = pd.read_csv("data_uji.csv")

log = log.tail(1)
X_ul = log.drop(['imgid','fold num'], axis=1)

print("\nTest dataset:-\n")
print(X_ul)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X,y)
pred = knn.predict(X_ul)
print("\nPrediksi : ")
if (pred == 0):
    print("Daun Sehat")
else:
    print("Daun Berpenyakit")

from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, pred))
accuracy_score(y_test, pred)


Training dataset:-

     feature1  feature2     feature3
0     39425.5       0.0   933.761535
1     39270.5       0.0   842.281306
2     36300.0       0.0   988.004176
3       213.5       0.0  1592.403232
4       196.0       0.0  1644.846884
5       181.0       0.0  1549.876321
6     29747.5       0.0   788.080296
7     31967.5       0.0   964.589962
8     36382.5       0.0  1032.874233
9       234.0       0.0  1762.469248
10    31259.5       0.0   881.477264
11      121.5       0.0   897.075244
12    33855.5       0.0   964.648837
13    37829.5       0.0   889.820410
14    34151.0       0.0   845.033613
15      204.5       0.0  1819.314920
16    36228.0       0.0   855.920916
17      215.5       0.0  1752.084472
18    37224.5       0.0  1022.388952
19      219.0       0.0  1693.900706
20      255.5       0.0  1853.765711
21    36942.0       0.0   964.430582
22      208.0       0.0  1838.871269
23      163.5       0.0  1495.751431
24    31229.0       0.0   913.376759
25    35263.5    

ValueError: Found input variables with inconsistent numbers of samples: [165, 1]

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=0)
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

[[67  7]
 [34 57]]


0.7515151515151515

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train,y_train)
y_pred1 = knn.predict(X_ul)
from sklearn.metrics import confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred1))
accuracy_score(y_test, y_pred1)

ValueError: Found input variables with inconsistent numbers of samples: [165, 1]